<a href="https://colab.research.google.com/github/vanesamarar/UW-LING-573-Group-5/blob/main/LING573_Train_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Authors: Nina Koh, Chelsea Kendrick, Vanesa Marar
# Install dependencies
# !pip install --quiet --upgrade pip
# !pip install --quiet --upgrade huggingface_hub
# !pip install --quiet torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --upgrade
# !pip install --quiet --upgrade torchao
# !pip install --quiet --upgrade accelerate
# !pip install --quiet --upgrade transformers
# !pip install --quiet sentence-transformers==2.2.2 fsspec==2023.6.0 numpy==1.26.4
# !pip install --quiet --upgrade transformers huggingface_hub sentence-transformers fsspec numpy rouge-metric nltk

# !pip install torch==2.0.1+cu118 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install torch==2.1.2+cu118 torchvision==0.16.2+cu118 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu118
# !pip install --quiet sentence-transformers==2.2.2 transformers==4.30.2 \
#  fsspec==2023.6.0 numpy==1.26.4 numba>=0.60.0
# !pip install --upgrade transformers huggingface-hub

# !pip install torch==2.0.1 transformers==4.37.2 accelerate==0.25.0 huggingface-hub==0.20.3
# !pip uninstall -y datasets fsspec huggingface-hub

# !pip install datasets==2.14.4 huggingface-hub==0.28.1 fsspec==2024.3.1

# !rm -rf ~/.cache/huggingface
# !rm -rf /root/.cache/huggingface
# !rm -rf /content/.cache/huggingface
# !rm -rf /content/hf_cache

# Uninstall everything nonessential
# !pip uninstall -y transformers accelerate huggingface-hub torch torchvision torchaudio \
#   tokenizers sentence-transformers peft fastai timm diffusers \
#   cudf-cu12 pylibcudf-cu12 tensorflow thinc \
#   torchtune dask-cudf-cu12 dopamine-rl cuml-cu12 tensorflow-decision-forests gradio spacy numpy


# !pip install numpy==1.24.4
# !pip install transformers==4.38.2 accelerate==0.27.2 huggingface-hub==0.28.1
# !pip install datasets rouge-metric nltk
# !pip install pyarrow==15.0.2 fsspec==2025.3.2


# import nltk
# nltk.download('punkt')

#!pip install torch==2.1.2+cu118 torchvision==0.16.2+cu118 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu118
# !pip install transformers==4.40.1 datasets==2.19.1 huggingface-hub==0.27.1 fsspec==2024.3.1 peft>=0.10.0
#!pip install -q \
#  transformers==4.35.2 \
#  datasets==2.19.1 \
#  huggingface-hub==0.27.1 \
#  peft>=0.5.0 \
#  evaluate rouge-score nltk
#!pip install --upgrade accelerate

!pip install -q \
  transformers==4.38.0 \
  datasets==2.19.1 \
  huggingface-hub==0.27.1 \
  peft==0.8.2 \
  evaluate rouge-score nltk

!pip install accelerate==0.27.2

# Don't use requirements.txt unless working in a local environment
# !pip install -r UW_LING_573_Group_5/requirements.txt

In [2]:
# Clone repo & set up environment
!git clone https://github.com/chelsk5/UW-LING-573-Group-5.git
!mv UW-LING-573-Group-5 UW_LING_573_Group_5 # Python modules use underscores

Cloning into 'UW-LING-573-Group-5'...
remote: Enumerating objects: 580, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 580 (delta 59), reused 72 (delta 27), pack-reused 457 (from 1)
Receiving objects: 100% (580/580), 446.88 KiB | 3.33 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [3]:
%cd UW_LING_573_Group_5
%ls
!git checkout chelsea-eval

/content/UW_LING_573_Group_5
data/       requirements.txt  summaries.json
Minutes.md  rouge_eval.py     tf-idf.py
README.md   summaries-gold/   UW-LING-573-Group-5/
Already on 'chelsea-eval'
Your branch is up to date with 'origin/chelsea-eval'.


In [4]:
# 1. Load data from local GitHub folder
import os, re, glob
from datasets import Dataset
from sklearn.model_selection import train_test_split

def clean_line(line):
    line = line.lower()
    line = re.sub(r'[^\w\s]', '', line)
    line = re.sub(r'\s+', ' ', line)
    return line

def load_local_opinosis(data_dir):
    examples = []
    search_path = os.path.join(data_dir, "*.txt.data")
    files = glob.glob(search_path)

    for file in files:
        topic = os.path.splitext(os.path.basename(file))[0].replace(".txt", "")
        with open(file, "r", encoding="utf-8", errors="ignore") as f:
            lines = [clean_line(line.strip()) for line in f if line.strip()]
            if lines:
                full_text = " ".join(lines)
                gold_summary = lines[0]  # TEMP: use first line as dummy summary
                examples.append({
                    "topic": topic,
                    "reviews": full_text,
                    "gold_summary": gold_summary
                })
    return examples

# Load data and split
data_dir = "/content/UW_LING_573_Group_5/data"
all_examples = load_local_opinosis(data_dir)

train_examples, test_examples = train_test_split(all_examples, test_size=0.2, random_state=1)

train_data = Dataset.from_list(train_examples)
test_data = Dataset.from_list(test_examples)


In [5]:
!ls /content/UW_LING_573_Group_5/data #check if data files are actually loading


accuracy_garmin_nuvi_255W_gps.txt.data
bathroom_bestwestern_hotel_sfo.txt.data
battery-life_amazon_kindle.txt.data
battery-life_ipod_nano_8gb.txt.data
battery-life_netbook_1005ha.txt.data
buttons_amazon_kindle.txt.data
comfort_honda_accord_2008.txt.data
comfort_toyota_camry_2007.txt.data
directions_garmin_nuvi_255W_gps.txt.data
display_garmin_nuvi_255W_gps.txt.data
eyesight-issues_amazon_kindle.txt.data
features_windows7.txt.data
fonts_amazon_kindle.txt.data
food_holiday_inn_london.txt.data
food_swissotel_chicago.txt.data
free_bestwestern_hotel_sfo.txt.data
gas_mileage_toyota_camry_2007.txt.data
interior_honda_accord_2008.txt.data
interior_toyota_camry_2007.txt.data
keyboard_netbook_1005ha.txt.data
location_bestwestern_hotel_sfo.txt.data
location_holiday_inn_london.txt.data
mileage_honda_accord_2008.txt.data
navigation_amazon_kindle.txt.data
parking_bestwestern_hotel_sfo.txt.data
performance_honda_accord_2008.txt.data
performance_netbook_1005ha.txt.data
price_amazon_kindle.txt.data
pri

In [5]:
# 2. REAL Preprocessing
def clean_data(example):
    """Preserves topic_id, reviews, and gold_summary."""
    return {
        "topic_id": example["topic"], # Keep the 'topic' key and rename it to 'topic_id' for clarity
        "reviews": example["reviews"],
        # We will overwrite gold_summary with the real one later
        "gold_summary": example["gold_summary"] # Keep the dummy summary for now if needed elsewhere, or remove if not
    }

# Apply mapping
cleaned_train = train_data.map(clean_data)
cleaned_test = test_data.map(clean_data)

# Print a couple of examples to check for 'topic_id'
print("Input reviews:")
print(cleaned_train[0]["reviews"])
print("\nTarget summary (dummy):") # Note this is the dummy summary
print(cleaned_train[0]["gold_summary"])
print("\nTopic ID:") # Check that topic_id is now present
print(cleaned_train[0]["topic_id"])


print("\nInput reviews:")
print(cleaned_train[1]["reviews"])
print("\nTarget summary (dummy):") # Note this is the dummy summary
print(cleaned_train[1]["gold_summary"])
print("\nTopic ID:") # Check that topic_id is now present
print(cleaned_train[1]["topic_id"])

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Input reviews:
i had to uninstall anti virus and selected other programs some of which did not have listings in the programs and features control panel section  this review briefly touches upon some of the key features and enhancements of microsofts latest os  all of these features combine to create a much improved and enjoyable desktop experience  i enjoyed the new convenience features in windows 7  they showed me pc laptops that had more features than the macbook and which cost way less than the macbook too  but that aside i like the availability of software the new features like the improved taskbar windows peek and the aero environment  if youre used to os x but want to come into the pc world where things cost less and you get more features by all means consider windows 7 the stepping stone to do just that  after seeing the pricing on the newest macbooks this week and the limited features compared to what i currently own i have to say that now is the right time to consider that swi

In [5]:
# 2. Preprocessing (not much to do since data is already cleaned)
def clean_data(example):
    """No-op cleaner — data already cleaned in loading step"""
    return {
        "reviews": example["reviews"],
        "gold_summary": example["gold_summary"]
    }

# Apply mapping
cleaned_train = train_data.map(clean_data)
cleaned_test = test_data.map(clean_data)

# Print a couple of examples
print("Input reviews:")
print(cleaned_train[0]["reviews"])
print("\nTarget summary:")
print(cleaned_train[0]["gold_summary"])

print("\nInput reviews:")
print(cleaned_train[1]["reviews"])
print("\nTarget summary:")
print(cleaned_train[1]["gold_summary"])

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Input reviews:
i had to uninstall anti virus and selected other programs some of which did not have listings in the programs and features control panel section  this review briefly touches upon some of the key features and enhancements of microsofts latest os  all of these features combine to create a much improved and enjoyable desktop experience  i enjoyed the new convenience features in windows 7  they showed me pc laptops that had more features than the macbook and which cost way less than the macbook too  but that aside i like the availability of software the new features like the improved taskbar windows peek and the aero environment  if youre used to os x but want to come into the pc world where things cost less and you get more features by all means consider windows 7 the stepping stone to do just that  after seeing the pricing on the newest macbooks this week and the limited features compared to what i currently own i have to say that now is the right time to consider that swi

In [6]:
# 3. Load tokenizer & model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype="float16",
#     bnb_4bit_quant_type="nf4",  # You can also use "fp4" depending on your needs
#     bnb_4bit_use_double_quant=True
# )

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possibl

In [7]:
# 4. REAL Tokenize reviews & gold summaries from Github repo

def load_gold_summary_for_review(topic_id, gold_dir="summaries-gold"):
    """Loads and combines .gold files for a given topic ID from the summaries-gold/ directory."""
    topic_path = os.path.join(gold_dir, topic_id)
    gold_summary = ""
    if os.path.exists(topic_path):
        for fname in os.listdir(topic_path):
            if fname.endswith(".gold"):
                with open(os.path.join(topic_path, fname), "r", encoding="utf-8") as f:
                    gold_summary += f.read().strip() + " "
    return gold_summary.strip()

def tokenize_data(cleaned_data, tokenizer):
    """Convert the raw text into token IDs for model processing using extracted gold summary."""
    topic_id = cleaned_data["topic_id"]
    reviews_text = cleaned_data["reviews"]

    gold_summary = load_gold_summary_for_review(topic_id)

    pref_text = "summarize: " + reviews_text

    tokenized_input = tokenizer(
        pref_text,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer([gold_summary], max_length=64, truncation=True)
    tokenized_input["labels"] = labels["input_ids"][0]

    return tokenized_input

# Example usage (assuming your data includes 'topic_id', 'reviews'):
tokenized_train = [tokenize_data(d, tokenizer) for d in cleaned_train]
tokenized_test = [tokenize_data(d, tokenizer) for d in cleaned_test]

# Check tokenization
print("Tokenized input example (input_ids):")
print(tokenized_train[0]["input_ids"])
print("Tokenized target (labels):")
print(tokenized_train[0]["labels"])


Tokenized input example (input_ids):
[21603, 10, 3, 23, 141, 12, 73, 7417, 1181, 6722, 11, 2639, 119, 1356, 128, 13, 84, 410, 59, 43, 11070, 16, 8, 1356, 11, 753, 610, 2952, 1375, 48, 1132, 16964, 13945, 1286, 128, 13, 8, 843, 753, 11, 15220, 7, 13, 3, 30574, 7, 1251, 3, 32, 7, 66, 13, 175, 753, 5148, 12, 482, 3, 9, 231, 3798, 11, 9231, 6555, 351, 3, 23, 2994, 8, 126, 6440, 753, 16, 3196, 489, 79, 3217, 140, 3, 102, 75, 4544, 7, 24, 141, 72, 753, 145, 8, 11486, 2567, 11, 84, 583, 194, 705, 145, 8, 11486, 2567, 396, 68, 24, 5915, 3, 23, 114, 8, 5576, 13, 889, 8, 126, 753, 114, 8, 3798, 2491, 1047, 3196, 20584, 11, 8, 15726, 1164, 3, 99, 39, 15, 261, 12, 3, 32, 7, 3, 226, 68, 241, 12, 369, 139, 8, 3, 102, 75, 296, 213, 378, 583, 705, 11, 25, 129, 72, 753, 57, 66, 598, 1099, 3196, 489, 8, 3, 19115, 3372, 12, 103, 131, 24, 227, 2492, 8, 5769, 30, 8, 3, 10909, 11486, 10467, 48, 471, 11, 8, 1643, 753, 3, 2172, 12, 125, 3, 23, 1083, 293, 3, 23, 43, 12, 497, 24, 230, 19, 8, 269, 97, 12, 1099, 

In [8]:
# 4. Tokenize reviews & gold summaries
def tokenize_data(cleaned_data):
    """Convert the raw text into token IDs for model processing"""
    pref_text = "summarize: " + cleaned_data["reviews"]

    tokenized_input = tokenizer(
        pref_text,
        max_length=511,
        truncation=True,
        padding="max_length"
    )

    gold_summary = cleaned_data["gold_summary"]
    labels = tokenizer([gold_summary], max_length=64, truncation=True)

    tokenized_input["labels"] = labels["input_ids"][0]
    return tokenized_input

# If you're using a list of cleaned dicts:
tokenized_train = [tokenize_data(d) for d in cleaned_train]
tokenized_test = [tokenize_data(d) for d in cleaned_test]

# Check tokenization
print("Tokenized input example (input_ids):")
print(tokenized_train[0]["input_ids"])
print("Tokenized target (labels):")
print(tokenized_train[0]["labels"])



Tokenized input example (input_ids):
[21603, 10, 3, 23, 141, 12, 73, 7417, 1181, 6722, 11, 2639, 119, 1356, 128, 13, 84, 410, 59, 43, 11070, 16, 8, 1356, 11, 753, 610, 2952, 1375, 48, 1132, 16964, 13945, 1286, 128, 13, 8, 843, 753, 11, 15220, 7, 13, 3, 30574, 7, 1251, 3, 32, 7, 66, 13, 175, 753, 5148, 12, 482, 3, 9, 231, 3798, 11, 9231, 6555, 351, 3, 23, 2994, 8, 126, 6440, 753, 16, 3196, 489, 79, 3217, 140, 3, 102, 75, 4544, 7, 24, 141, 72, 753, 145, 8, 11486, 2567, 11, 84, 583, 194, 705, 145, 8, 11486, 2567, 396, 68, 24, 5915, 3, 23, 114, 8, 5576, 13, 889, 8, 126, 753, 114, 8, 3798, 2491, 1047, 3196, 20584, 11, 8, 15726, 1164, 3, 99, 39, 15, 261, 12, 3, 32, 7, 3, 226, 68, 241, 12, 369, 139, 8, 3, 102, 75, 296, 213, 378, 583, 705, 11, 25, 129, 72, 753, 57, 66, 598, 1099, 3196, 489, 8, 3, 19115, 3372, 12, 103, 131, 24, 227, 2492, 8, 5769, 30, 8, 3, 10909, 11486, 10467, 48, 471, 11, 8, 1643, 753, 3, 2172, 12, 125, 3, 23, 1083, 293, 3, 23, 43, 12, 497, 24, 230, 19, 8, 269, 97, 12, 1099, 

In [8]:
# 5. Create batch of examples
from transformers import DataCollatorForSeq2Seq

# Dynamically pad sentences to the longest length
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
#60. Evaluate model using HuggingFace

# Copyright 2020 The HuggingFace Evaluate Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" ROUGE metric from Google Research github repo. """

# The dependencies in https://github.com/google-research/google-research/blob/master/rouge/requirements.txt
import absl  # Here to have a nice missing dependency error message early on
import datasets
import nltk  # Here to have a nice missing dependency error message early on
import numpy  # Here to have a nice missing dependency error message early on
import six  # Here to have a nice missing dependency error message early on
from rouge_score import rouge_scorer, scoring

import evaluate


_CITATION = """\
@inproceedings{lin-2004-rouge,
    title = "{ROUGE}: A Package for Automatic Evaluation of Summaries",
    author = "Lin, Chin-Yew",
    booktitle = "Text Summarization Branches Out",
    month = jul,
    year = "2004",
    address = "Barcelona, Spain",
    publisher = "Association for Computational Linguistics",
    url = "https://www.aclweb.org/anthology/W04-1013",
    pages = "74--81",
}
"""

_DESCRIPTION = """\
ROUGE, or Recall-Oriented Understudy for Gisting Evaluation, is a set of metrics and a software package used for
evaluating automatic summarization and machine translation software in natural language processing.
The metrics compare an automatically produced summary or translation against a reference or a set of references (human-produced) summary or translation.

Note that ROUGE is case insensitive, meaning that upper case letters are treated the same way as lower case letters.

This metrics is a wrapper around Google Research reimplementation of ROUGE:
https://github.com/google-research/google-research/tree/master/rouge
"""

_KWARGS_DESCRIPTION = """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"\n"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Returns:
    rouge1: rouge_1 (f1),
    rouge2: rouge_2 (f1),
    rougeL: rouge_l (f1),
    rougeLsum: rouge_lsum (f1)
Examples:

    >>> rouge = evaluate.load('rouge')
    >>> predictions = ["hello there", "general kenobi"]
    >>> references = ["hello there", "general kenobi"]
    >>> results = rouge.compute(predictions=predictions, references=references)
    >>> print(results)
    {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
"""


class Tokenizer:
    """Helper class to wrap a callable into a class with a `tokenize` method as used by rouge-score."""

    def __init__(self, tokenizer_func):
        self.tokenizer_func = tokenizer_func

    def tokenize(self, text):
        return self.tokenizer_func(text)


@evaluate.utils.file_utils.add_start_docstrings(_DESCRIPTION, _KWARGS_DESCRIPTION)
class Rouge(evaluate.Metric):
    def _info(self):
        return evaluate.MetricInfo(
            description=_DESCRIPTION,
            citation=_CITATION,
            inputs_description=_KWARGS_DESCRIPTION,
            features=[
                datasets.Features(
                    {
                        "predictions": datasets.Value("string", id="sequence"),
                        "references": datasets.Sequence(datasets.Value("string", id="sequence")),
                    }
                ),
                datasets.Features(
                    {
                        "predictions": datasets.Value("string", id="sequence"),
                        "references": datasets.Value("string", id="sequence"),
                    }
                ),
            ],
            codebase_urls=["https://github.com/google-research/google-research/tree/master/rouge"],
            reference_urls=[
                "https://en.wikipedia.org/wiki/ROUGE_(metric)",
                "https://github.com/google-research/google-research/tree/master/rouge",
            ],
        )

    def _compute(
        self, predictions, references, rouge_types=None, use_aggregator=True, use_stemmer=False, tokenizer=None
    ):
        if rouge_types is None:
            rouge_types = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

        multi_ref = isinstance(references[0], list)

        if tokenizer is not None:
            tokenizer = Tokenizer(tokenizer)

        scorer = rouge_scorer.RougeScorer(rouge_types=rouge_types, use_stemmer=use_stemmer, tokenizer=tokenizer)
        if use_aggregator:
            aggregator = scoring.BootstrapAggregator()
        else:
            scores = []

        for ref, pred in zip(references, predictions):
            if multi_ref:
                score = scorer.score_multi(ref, pred)
            else:
                score = scorer.score(ref, pred)
            if use_aggregator:
                aggregator.add_scores(score)
            else:
                scores.append(score)

        if use_aggregator:
            result = aggregator.aggregate()
            for key in result:
                result[key] = result[key].mid.fmeasure

        else:
            result = {}
            for key in scores[0]:
                result[key] = list(score[key].fmeasure for score in scores)

        return result

In [ ]:
# 6A. Evaluate model using HuggingFace
!pip install rouge-score evaluate

import numpy as np
#from UW_LING_573_Group_5 import rouge
#from rouge import rouge_scorer
from nltk import sent_tokenize

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

def insert_boundaries(text):
  """Inserts newline characters between each sentence in the given text"""
  sentences = sent_tokenize(text)
  return "\n".join(sentences)

def compute_rouge(eval_pred):
  """Decodes token IDs and computes the ROUGE scores of generated summaries"""
  predictions, labels = eval_pred # model outputs & gold summaries as token IDs

  predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
  # Decode model output IDs back into text
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

  # Replace padding tokens IDs (-100) with tokenizer's pad token ID (usually 0)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  # Decode gold summary IDs back into text
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # Compute precision, recall, & f1 across ROUGE scores
  rouge1_p, rouge1_r, rouge1_f = [], [], []
  rouge2_p, rouge2_r, rouge2_f = [], [], []
  rougeL_p, rougeL_r, rougeL_f = [], [], []

  for pred, label in zip(decoded_preds, decoded_labels):
    scores = scorer.score(pred, label)
    # ROUGE-1
    rouge1_p.append(scores["rouge1"].precision)
    rouge1_r.append(scores["rouge1"].recall)
    rouge1_f.append(scores["rouge1"].fmeasure)

    # ROUGE-2
    rouge2_p.append(scores["rouge2"].precision)
    rouge2_r.append(scores["rouge2"].recall)
    rouge2_f.append(scores["rouge2"].fmeasure)

    # ROUGE-L
    rougeL_p.append(scores["rougeL"].precision)
    rougeL_r.append(scores["rougeL"].recall)
    rougeL_f.append(scores["rougeL"].fmeasure)

  # print("\n--- ORIGINAL ---")
  # print("Prediction:", decoded_preds[0])
  # print("Gold:", decoded_labels[0])

  # print("\n--- HF-style Preprocessed ---")
  # print("Pred:", preprocess(decoded_preds[0]))
  # print("Gold:", preprocess(decoded_labels[0]))

  # Return rounded mean across scores
  return {
      "rouge1_precision": round(np.mean(rouge1_p), 4),
      "rouge1_recall": round(np.mean(rouge1_r), 4),
      "rouge1_f1": round(np.mean(rouge1_f), 4),
      "rouge2_precision": round(np.mean(rouge2_p), 4),
      "rouge2_recall": round(np.mean(rouge2_r), 4),
      "rouge2_f1": round(np.mean(rouge2_f), 4),
      "rougeL_precision": round(np.mean(rougeL_p), 4),
      "rougeL_recall": round(np.mean(rougeL_r), 4),
      "rougeL_f1": round(np.mean(rougeL_f), 4),
  }

# The code below is only useable if this ROUGE runtime error gets resolved:
# https://huggingface.co/evaluate-metric?sort_spaces=modified#spaces

# import evaluate

# rouge = evaluate.load("rouge")

# def compute_rouge(eval_pred):
#   """Decodes token IDs and computes the ROUGE score of generated summaries"""
#   predictions, labels = eval_pred # model outputs & gold summaries as token IDs
#   # Decode model output IDs back into text
#   decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

#   # Replace padding tokens IDs (-100) with tokenizer's pad token ID (usually 0)
#   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#   # Decode gold summary IDs back into text
#   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#   # Compare model output against gold labels via ROUGE
#   result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#   clean_result = {k: round(v, 4) for k,v in result.items()} # map rouge type to value
#   return clean_result

In [13]:
# 6B. Alternative ROUGE evaluation using rouge_metric PyRouge
# This implementation currently yields inflated scores due to internal
# scoring system (not comparable to HuggingFace)
!pip install rouge-metric

import numpy as np
from rouge_metric import PyRouge
import re
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

rouge = PyRouge(
    rouge_n=(1, 2, 4),
    rouge_l=True,
    rouge_w=True,
    rouge_s=True,
    rouge_su=True,
    skip_gap=4
)

def preprocess(text):
#  text = text.lower()
#  text = re.sub(r'[^\w\s]', '', text)
#  tokens = text.split()
#  stemmed = [stemmer.stem(t) for t in tokens]
#  return ' '.join(stemmed)
   return text.strip().split() # apply minimal preprocessing

def compute_rouge(eval_pred):
  predictions, labels = eval_pred
  # NEW LINE!!!
  predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  hypotheses = [preprocess(pred) for pred in decoded_preds]
  references = [[preprocess(label)] for label in decoded_labels]

  #scores = rouge.get_scores(hypotheses, references, avg=True)
  scores = rouge.evaluate_tokenized(hypotheses, references)

  print("\n--- ORIGINAL ---")
  print("Prediction:", decoded_preds[0])
  print("Gold:", decoded_labels[0])

  print("\n--- PyROUGE-style Preprocessed ---")
  print("Pred:", decoded_preds[0])
  print("Gold:", decoded_labels[0])

  return {
      "rouge1_precision": round(scores["rouge-1"]["p"], 4),
      "rouge1_recall": round(scores["rouge-1"]["r"], 4),
      "rouge1_f1": round(scores["rouge-1"]["f"], 4),
      "rouge2_precision": round(scores["rouge-2"]["p"], 4),
      "rouge2_recall": round(scores["rouge-2"]["r"], 4),
      "rouge2_f1": round(scores["rouge-2"]["f"], 4),
      "rougeL_precision": round(scores["rouge-l"]["p"], 4),
      "rougeL_recall": round(scores["rouge-l"]["r"], 4),
      "rougeL_f1": round(scores["rouge-l"]["f"], 4),
      "rougeSU4_precision": round(scores["rouge-su4"]["p"], 4),
      "rougeSU4_recall": round(scores["rouge-su4"]["r"], 4),
      "rougeSU4_f1": round(scores["rouge-su4"]["f"], 4),
  }

In [14]:
# 7. Train model
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
from accelerate import Accelerator

# Manually tune hyperparameters

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-small-results", # folder to save progress
    evaluation_strategy="epoch", # evaluate after one complete pass
    learning_rate=3e-5,
    per_device_train_batch_size=8, # small batch size for small data
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3, # number of checkpoints (subject to change?)
    num_train_epochs=5, # total passes thru data (also stc...)
    generation_max_length=100,
    predict_with_generate=True,
    fp16=False,# changed to False to avoid ValueError
    report_to="none"
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_rouge
)

# Train (finally)
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1 Precision,Rouge1 Recall,Rouge1 F1,Rouge2 Precision,Rouge2 Recall,Rouge2 F1,Rougel Precision,Rougel Recall,Rougel F1,Rougesu4 Precision,Rougesu4 Recall,Rougesu4 F1
1,No log,3.644899,0.780900,0.757300,0.768900,0.445800,0.432200,0.438900,0.778000,0.754800,0.766200,0.603400,0.582800,0.592900
2,No log,3.604603,0.725200,0.784200,0.753600,0.417700,0.451200,0.433800,0.721900,0.780700,0.750100,0.556200,0.600500,0.577500
3,No log,3.581417,0.676000,0.817600,0.740100,0.390700,0.476900,0.429500,0.672100,0.812400,0.735600,0.521300,0.634900,0.572500
4,No log,3.568949,0.655000,0.834500,0.733900,0.370800,0.480600,0.418600,0.648600,0.826900,0.727000,0.497200,0.642300,0.560500
5,No log,3.564662,0.655000,0.834500,0.733900,0.370800,0.480600,0.418600,0.648600,0.826900,0.727000,0.497200,0.642300,0.560500



--- ORIGINAL ---
Prediction: the camry is a comfortable car and has good road manner. the cabin is comfortable and comfortable. the ride is quiet and comfortable. The seats arent comfortable. The interior is comfortable. The interior is comfortable. The interior is comfortable. The interior is comfortable. The interior is comfortable.
Gold: Its very comfortable for long trips. The Camry offers interior comfort, while providing a quiet ride. Comfortable seating and easy to drive. Overall very comfortable ride front and back. Nice and roomy. Its very comfortable and a quiet ride with low levels of noise. Overall very comfortable ride

--- PyROUGE-style Preprocessed ---
Pred: the camry is a comfortable car and has good road manner. the cabin is comfortable and comfortable. the ride is quiet and comfortable. The seats arent comfortable. The interior is comfortable. The interior is comfortable. The interior is comfortable. The interior is comfortable. The interior is comfortable.
Gold: Its

TrainOutput(global_step=25, training_loss=3.3024807739257813, metrics={'train_runtime': 594.9767, 'train_samples_per_second': 0.336, 'train_steps_per_second': 0.042, 'total_flos': 27068360294400.0, 'train_loss': 3.3024807739257813, 'epoch': 5.0})

In [11]:
# 8. Log metrics to JSON file
metrics = trainer.evaluate()
print(metrics) # sanity check

import json
with open("metrics.json", "w") as f:
  json.dump(metrics, f, indent=2)


--- ORIGINAL ---
Prediction: the camry is a comfortable car and has good road manner. it is quiet and comfortable and has good road manner. the cabin is comfortable and comfortable. the car is quiet and comfortable. the engine isnt strong at all. the car doesnt deliver good comfort while driving it. the car is quiet and comfortable. the car doesnt deliver good comfort while driving it. it is quiet and comfortable. the engine isnt strong at
Gold: Its very comfortable for long trips. The Camry offers interior comfort, while providing a quiet ride. Comfortable seating and easy to drive. Overall very comfortable ride front and back. Nice and roomy. Its very comfortable and a quiet ride with low levels of noise. Overall very comfortable ride

--- PyROUGE-style Preprocessed ---
Pred: the camry is a comfortable car and has good road manner. it is quiet and comfortable and has good road manner. the cabin is comfortable and comfortable. the car is quiet and comfortable. the engine isnt strong 

In [14]:
# 10. Save predictions & labels in json files
import os, json
from datasets import concatenate_datasets
from torch.utils.data import DataLoader
from transformers import DataCollatorForSeq2Seq
import torch
import numpy as np

# Combine tokenized train and test sets
tokenized_train_ds = Dataset.from_list(tokenized_train)
tokenized_test_ds = Dataset.from_list(tokenized_test)
tokenized_opinosis = concatenate_datasets([tokenized_train_ds, tokenized_test_ds])
print(tokenized_opinosis.column_names)


# Remove raw text fields (adjust names as needed)
try:
    tokenized_opinosis = tokenized_opinosis.remove_columns(
        ["review_sents", "summaries", "reviews", "gold_summary"]
    )
except ValueError as e:
    print(f"Warning: Could not remove columns. Check column names. Error: {e}")
    # You might want to inspect the column names here
    print("Current columns:", tokenized_opinosis.column_names)

# Create DataLoader with automatic padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
dataloader = DataLoader(tokenized_opinosis, batch_size=8, collate_fn=data_collator)

# Prepare model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

decoded_preds = []

# Generate summaries
for batch in dataloader:
    # Ensure batch is moved to the correct device
    batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=100,
            num_beams=4,
            early_stopping=True
        )

    decoded_batch = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    decoded_preds.extend([d.strip() for d in decoded_batch])

# Save generated summaries
with open("t5_summaries.json", "w") as f:
    json.dump(decoded_preds, f, indent=2)

# Decode reference summaries from trainer (just for labels)
predictions = trainer.predict(tokenized_opinosis)
labels = np.where(predictions.label_ids != -100, predictions.label_ids, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
decoded_labels = [label.strip() for label in decoded_labels]

# Save reference summaries
with open("reference_summaries.json", "w") as f:
    json.dump(decoded_labels, f, indent=2)

# import os, json
# from collections import defaultdict
# from datasets import concatenate_datasets

# # Model-generated summaries
# tokenized_opinosis = concatenate_datasets([tokenized_train, tokenized_test]) # full dataset
# predictions = trainer.predict(tokenized_opinosis, max_length=100) # use full dataset, not just training
# pred_ids = np.argmax(predictions.predictions, axis=-1)
# decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

# with open("t5_summaries.json", "w") as f:
#     json.dump(decoded_preds, f, indent=2)

# # Reference summaries
# labels = np.where(predictions.label_ids != -100, predictions.label_ids, tokenizer.pad_token_id)
# decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

# with open("reference_summaries.json", "w") as f:
#     json.dump(decoded_labels, f, indent=2)

# system_dir = "t5-small-results/system_summaries" # t5 predictions
# model_dir = "t5-small-results/model_summaries" # gold reference
# os.makedirs(system_dir, exist_ok=True)
# os.makedirs(model_dir, exist_ok=True)

# Write to output files, where each file contains a single summary
# for i, (pred, ref) in enumerate(zip(decoded_preds, decoded_labels)):
#     with open(f"{system_dir}/{i}.txt", "w") as sys_f:
#         sys_f.write(pred.strip() + "\n")
#     with open(f"{model_dir}/{i}.A.1.txt", "w") as ref_f:
#         ref_f.write(ref.strip() + "\n")

['input_ids', 'attention_mask', 'labels']
Current columns: ['input_ids', 'attention_mask', 'labels']



--- ORIGINAL ---
Prediction: i had to uninstall anti virus and selected other programs some of which did not have listings in the programs and features control panel section this review briefly touches upon some of the key features and enhancements of microsofts latest os all these features combine to create a much improved and enjoyable desktop experience. i enjoyed the new convenience features in windows 7 they showed me pc laptops that had more features than the macbook and which cost way less than the macbook
Gold: i had to uninstall anti virus and selected other programs some of which did not have listings in the programs and features control panel section

--- PyROUGE-style Preprocessed ---
Pred: i had to uninstall anti virus and selected other programs some of which did not have listings in the programs and features control panel section this review briefly touches upon some of the key features and enhancements of microsofts latest os all these features combine to create a much